In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
import fancyimpute
import missingno as msno


%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/donggaocun_processed.feather


In [10]:
import math 
import re


# From fast.ai 0.7
def add_datepart(df, fldname, drop=True, time=False):
    "Helper function that adds columns relevant to a date."
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)
        
        
def timeseries_to_supervised(df, lag=1, dropnan=True):
    
    df = pd.DataFrame(df)
    columns = [df.shift(i).rename(columns=lambda x: f"{x}(-{i})") for i in range(1, lag+1)]
    columns.insert(0, df)
    df = pd.concat(columns, axis=1)
    
    if dropnan:
        df.dropna(inplace=True)
    else:
        df = df[lag:] #skip nan at the start
        df.fillna(-1, inplace=True)
        
    return df

def split_vals(a,n): return a[:n].copy(), a[n:].copy()

def generate_train_test_data(df, target_column, split_ratio=None, split_date=None):
    
#     supervised_df = timeseries_to_supervised(df, lag_observations, dropnan=dropnan)

    if split_ratio is not None:
        split_index = int(len(df) * split_ratio)
    elif split_date is not None:
        date_index = df.index.get_loc(split_date)
        if type(date_index) == slice:
            split_index = date_index.start
        else:
            split_index = date_index
    else:
        raise("Must specify split ratio or split date")
    
    non_lagged_columns = list(df.columns[~df.columns.str.contains('\(.*\)')]) # lagged columns are labeled (t-1)
    non_lagged_columns.append(target_column)
    X = df.drop(non_lagged_columns, axis=1)
    y = df[target_column]
    
    
    X_train, X_valid = split_vals(X, split_index)
    y_train, y_valid = split_vals(y, split_index)
       
    
    return X_train, y_train, X_valid, y_valid

def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def smape(actual, predicted):
    dividend= np.abs(np.array(actual) - np.array(predicted))
    denominator = np.array(actual) + np.array(predicted)
    
    return 2 * np.mean(np.divide(dividend, denominator, out=np.zeros_like(dividend), where=denominator!=0, casting='unsafe'))


def print_score(m, X_train, y_train, X_valid, y_valid):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [11]:

donggaocun_aq = pd.read_feather('/kaggle/input/donggaocun_processed.feather')
donggaocun_aq.head()

/opt/conda/lib/python3.6/site-packages/pandas/io/feather_format.py:117: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_feather(path, columns=columns, nthreads=int_use_threads)
/opt/conda/lib/python3.6/site-packages/pyarrow/pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


,utc_datetime,PM2.5,PM10,NO2,CO,O3,SO2,beijing_grid_452_temperature,beijing_grid_452_pressure,beijing_grid_452_humidity,...,pinggu_meo_weather_Cloudy,pinggu_meo_weather_Dust,pinggu_meo_weather_Fog,pinggu_meo_weather_Hail,pinggu_meo_weather_Haze,pinggu_meo_weather_Light Rain,pinggu_meo_weather_Rain,pinggu_meo_weather_Sand,pinggu_meo_weather_Snow,pinggu_meo_weather_Sunny/clear
0,2017-01-30 16:00:00,57.0,NaN,10.0,0.7,54.0,14.0,-6.17,1015.44,14.58,...,0,0,0,0,1,0,0,0,0,0
1,2017-01-30 17:00:00,61.0,NaN,10.0,0.7,47.0,15.0,-6.35,1015.06,15.41,...,0,0,0,0,1,0,0,0,0,0
2,2017-01-30 18:00:00,67.0,NaN,6.0,0.7,43.0,13.0,-6.53,1014.68,16.24,...,0,0,0,0,1,0,0,0,0,0
3,2017-01-30 19:00:00,74.0,NaN,7.0,0.9,42.0,13.0,-6.96,1014.20,17.12,...,0,0,0,0,1,0,0,0,0,0
4,2017-01-30 20:00:00,69.0,NaN,8.0,0.9,43.0,15.0,-7.39,1013.72,18.00,...,0,0,0,0,0,0,0,0,0,1


## Tuning XGBoost Parameters

In [12]:
class BlockingTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits
    
    def get_n_splits(self, X, y, groups):
        return self.n_splits
    
    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)

        margin = 0
        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = int(0.8 * (stop - start)) + start
            yield indices[start: mid], indices[mid + margin: stop]

In [13]:
pm25_supervised_df = timeseries_to_supervised(donggaocun_aq.set_index('utc_datetime'), 16, dropnan=False)
pm10_supervised_df = timeseries_to_supervised(donggaocun_aq.set_index('utc_datetime'), 15, dropnan=False)
O3_supervised_df = timeseries_to_supervised(donggaocun_aq.set_index('utc_datetime'), 10, dropnan=False)

X_train_pm25, y_train_pm25, X_valid_pm25, y_valid_pm25 = generate_train_test_data(pm25_supervised_df, split_ratio=0.8, target_column='PM2.5')
X_train_pm10, y_train_pm10, X_valid_pm10, y_valid_pm10 = generate_train_test_data(pm10_supervised_df, split_ratio=0.8, target_column='PM10')
X_train_O3, y_train_O3, X_valid_O3, y_valid_O3 = generate_train_test_data(O3_supervised_df, split_ratio=0.8, target_column='O3')

In [14]:
X_train_pm25.head()

,PM2.5(-1),PM10(-1),NO2(-1),CO(-1),O3(-1),SO2(-1),beijing_grid_452_temperature(-1),beijing_grid_452_pressure(-1),beijing_grid_452_humidity(-1),beijing_grid_452_wind_direction(-1),...,pinggu_meo_weather_Cloudy(-16),pinggu_meo_weather_Dust(-16),pinggu_meo_weather_Fog(-16),pinggu_meo_weather_Hail(-16),pinggu_meo_weather_Haze(-16),pinggu_meo_weather_Light Rain(-16),pinggu_meo_weather_Rain(-16),pinggu_meo_weather_Sand(-16),pinggu_meo_weather_Snow(-16),pinggu_meo_weather_Sunny/clear(-16)
utc_datetime,,,,,,,,,,,,,,,,,,,,,
2017-01-31 08:00:00,40.0,-1.0,5.0,0.7,72.0,18.0,-0.10,1010.38,10.09,289.43,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2017-01-31 09:00:00,45.0,-1.0,4.0,0.8,70.0,18.0,-0.88,1010.39,11.76,257.21,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2017-01-31 10:00:00,48.0,-1.0,5.0,0.7,63.0,16.0,-1.67,1010.40,13.43,210.99,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2017-01-31 11:00:00,55.0,-1.0,7.0,0.8,60.0,16.0,-2.12,1011.10,13.42,259.81,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2017-01-31 12:00:00,55.0,-1.0,7.0,0.8,58.0,17.0,-2.58,1011.79,13.40,310.38,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
# n_estimators = [100, 200, 300, 400, 500]
# learning_rate = [0.0001, 0.001, 0.01, 0.1]
# max_depth = [2, 4, 6, 8]

# model = XGBRegressor()
# param_grid = dict(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate)
# btscv = BlockingTimeSeriesSplit(n_splits=5)
# grid_search = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', n_jobs=-1, cv=btscv, verbose=1)
# grid_result = grid_search.fit(X_train_pm25, y_train_pm25)

In [ ]:
# grid_result.best_params_

### Test XGBoost parameters

In [ ]:
# {'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 500}
pm25_xg_model = XGBRegressor(n_estimators=500, learning_rate=0.01, max_depth=2)
pm25_xg_model.fit(X_train_pm25, y_train_pm25)

pm25_non_na_values = y_valid_pm25 != -1
y_valid_pm25 = y_valid_pm25[pm25_non_na_values]
X_valid_pm25 = X_valid_pm25[pm25_non_na_values]

print_score(pm25_xg_model, X_train_pm25, y_train_pm25, X_valid_pm25, y_valid_pm25)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 10))
plot_importance(pm25_xg_model, height=0.9, max_num_features=10, ax=ax)

In [ ]:
# Save xgboost pm2.5 results

pm25_predictions = pm25_xg_model.predict(X_valid_pm25)
np.save('pm25_xg_predictions', pm25_predictions)
y_valid_pm25.to_csv('pm25_actual.csv')

In [ ]:
pm10_xg_model = XGBRegressor(n_estimators=500, learning_rate=0.01, max_depth=2)
pm10_xg_model.fit(X_train_pm10, y_train_pm10)

pm10_non_na_values = y_valid_pm10 != -1
y_valid_pm10 = y_valid_pm10[pm10_non_na_values]
X_valid_pm10 = X_valid_pm10[pm10_non_na_values]


print_score(pm10_xg_model, X_train_pm10, y_train_pm10, X_valid_pm10, y_valid_pm10)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 10))
plot_importance(pm10_xg_model, height=0.9, max_num_features=10, ax=ax)

In [ ]:
pm10_xg_predictions = pm10_xg_model.predict(X_valid_pm10)
np.save('pm10_xg_predictions', pm10_xg_predictions)
y_valid_pm10.to_csv('pm10_actual.csv')

In [ ]:
O3_xg_model = XGBRegressor(n_estimators=500, learning_rate=0.01, max_depth=2)
O3_xg_model.fit(X_train_O3, y_train_O3)

O3_non_na_values = y_valid_O3 != -1
y_valid_O3 = y_valid_O3[O3_non_na_values]
X_valid_O3 = X_valid_O3[O3_non_na_values]

print_score(O3_xg_model, X_train_O3, y_train_O3, X_valid_O3, y_valid_O3)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 10))
plot_importance(O3_xg_model, height=0.9, max_num_features=10, ax=ax)

In [ ]:
O3_xg_predictions = O3_xg_model.predict(X_valid_O3)
np.save('O3_xg_predictions', O3_xg_predictions)
y_valid_O3.to_csv('O3_actual.csv')

## Tuning lightgbm Parameters

In [ ]:
# lgb_grid_params = {
#     'learning_rate': [0.001, 0.01, 0.04, 0.07, 0.1],
#     'n_estimators': [50, 100, 150, 200, 300],
#     'num_leaves': [6, 8,12,16],
#     'boosting_type' : ['gbdt'],
#     'min_data_in_leaf' : [30, 50, 80, 100],
#     }

# lgb_model = lgb.LGBMRegressor()
# lgb_btscv = BlockingTimeSeriesSplit(n_splits=5)
# grid_search_lgb = GridSearchCV(lgb_model, lgb_grid_params, scoring='neg_mean_squared_error', n_jobs=-1, cv=lgb_btscv, verbose=1)
# grid_result = grid_search_lgb.fit(X_train_pm25, y_train_pm25)

In [ ]:
# grid_result.best_params_

In [ ]:
# lightgbm best params
# {'boosting_type': 'gbdt',
#  'learning_rate': 0.1,
#  'min_data_in_leaf': 30,
#  'n_estimators': 300,
#  'num_leaves': 6}


pm25_lgb_model = lgb.LGBMRegressor(boosting_type='gbdt', learning_rate=0.1, min_data_in_leaf=30, n_estimators=300, num_leaves=6)
pm25_lgb_model.fit(X_train_pm25, y_train_pm25)

print_score(pm25_lgb_model, X_train_pm25, y_train_pm25, X_valid_pm25, y_valid_pm25)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 10))
lgb.plot_importance(pm25_lgb_model, height=0.9, max_num_features=10, ax=ax)

In [ ]:
pm25_lgb_predictions = pm25_lgb_model.predict(X_valid_pm25)
np.save('pm25_lgb_predictions', pm25_lgb_predictions)

In [ ]:
pm10_lgb_model = lgb.LGBMRegressor(boosting_type='gbdt', learning_rate=0.1, min_data_in_leaf=30, n_estimators=300, num_leaves=6)
pm10_lgb_model.fit(X_train_pm10, y_train_pm10)

print_score(pm10_lgb_model, X_train_pm10, y_train_pm10, X_valid_pm10, y_valid_pm10)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 10))
lgb.plot_importance(pm10_lgb_model, height=0.9, max_num_features=10, ax=ax)

In [ ]:
pm10_lgb_predictions = pm10_lgb_model.predict(X_valid_pm10)
np.save('pm10_lgb_predictions', pm10_lgb_predictions)

In [ ]:
O3_lgb_model = lgb.LGBMRegressor(boosting_type='gbdt', learning_rate=0.1, min_data_in_leaf=30, n_estimators=300, num_leaves=6)
O3_lgb_model.fit(X_train_O3, y_train_O3)

print_score(O3_lgb_model, X_train_O3, y_train_O3, X_valid_O3, y_valid_O3)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 10))
lgb.plot_importance(O3_lgb_model, height=0.9, max_num_features=10, ax=ax)

In [ ]:
O3_lgb_predictions = O3_lgb_model.predict(X_valid_O3)
np.save('O3_lgb_predictions', O3_lgb_predictions)